In [1]:
from model_v2 import build_lm_classifier_tagger_inference, LSTM_SAVED_STATE
from utils import get_batch_classifier_inference, clean_text
import json
import tensorflow as tf
from tensorflow.contrib.cudnn_rnn.python.ops import cudnn_rnn_ops
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
%matplotlib inline

In [2]:
with open('109_Bank/checkpoints/model_configs.json', 'r') as inp:
    lm_params = json.load(inp)
with open('109_Bank/checkpoints/classifier_configs.json') as inp:
    cls_params = json.load(inp)
with open('109_Bank/checkpoints/tagger_configs.json') as inp:
    tag_params = json.load(inp)
language_model, classifier, tagger = build_lm_classifier_tagger_inference(lm_params, 
                                                                          cls_params, tag_params, 
                                                                          is_cpu=False)

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
# Load saver in GPU to restore weights
gpu_saver = tf.train.Saver([x for x in tf.global_variables() if x not in tf.get_collection(LSTM_SAVED_STATE)])
gpu_saver.restore(session, '109_Bank/checkpoints/chatbot_test/model.cpkt-723')

INFO:tensorflow:Restoring parameters from 109_Bank/checkpoints/chatbot_test/model.cpkt-723


In [4]:
def cudnn_lstm_to_lstmblockcell(input_size, n_units, weights):
    weights, biases = cudnn_rnn_ops.cudnn_rnn_opaque_params_to_canonical(
                rnn_mode='lstm', 
                num_layers=1, 
                num_units=n_units,
                input_size=input_size, 
                params=weights, 
                input_mode='linear_input',
                direction='unidirectional'
    )
    weights = [tf.concat((w, r), axis=1) for w, r in zip(weights[:4], weights[4:])]
    biases = [tf.add(w, r) for w, r in zip(biases[:4], biases[4:])]
    # Cudnn LSTM weights are in ifco order, other tf LSTMs are in icfo order.
    new_order = [0, 2, 1, 3]
    weights = tf.concat([weights[i] for i in new_order], axis=0)
    biases = tf.concat([biases[i] for i in new_order], axis=0)
    return tf.transpose(weights), tf.transpose(biases)

In [5]:
name2var = {}
assign_ops = []
for v in tf.global_variables():
    if 'cudnn_lstm' in v.op.name:
        prefix = v.op.name.replace('cudnn_lstm', 'cudnn_compatible_lstm_cell').replace('opaque_kernel', '')
        idx = re.search('cudnn_lstm(_\d+)?', v.op.name).group(0)
        idx = re.search('\d+', idx)
        if idx is None:
            idx = 0
        else:
            idx = int(idx.group(0))
        n_units = lm_params['rnn_layers'][idx]['units']
        input_shape = lm_params.get('projection_dims', None)
        if input_shape is None:
            if idx == 0:
                input_shape = sum(x for _,x in lm_params['char_cnn_options']['layers'])
            else:
                input_shape = lm_params['rnn_layers'][idx-1]['units']
        kernel_name = prefix + 'kernel'
        bias_name = prefix + 'bias'
        kernel_val, bias_val = cudnn_lstm_to_lstmblockcell(input_shape, n_units, v)
        name2var[kernel_name] = tf.get_variable(kernel_name, shape=(input_shape+n_units, 4*n_units), dtype=tf.float32)
        name2var[bias_name] = tf.get_variable(bias_name, shape=(4*n_units,), dtype=tf.float32)
        assign_ops.append(tf.assign(name2var[kernel_name], kernel_val))
        assign_ops.append(tf.assign(name2var[bias_name], bias_val))
        
    else:
        name2var[v.op.name] = v
assign_ops = tf.group(assign_ops)

In [6]:
session.run(assign_ops)

In [7]:
# Create CPU saver
cpu_saver = tf.train.Saver(name2var)
cpu_saver.save(session, '109_Bank/checkpoints/chatbot_cpu/model.cpkt-723', write_meta_graph=False)
session.close()

In [8]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
tf.reset_default_graph()
language_model, classifier, tagger = build_lm_classifier_tagger_inference(lm_params, 
                                                                          cls_params, tag_params, 
                                                                          is_cpu=True)

In [9]:
cpu_saver = tf.train.Saver(name2var)
cpu_saver.restore(session, '109_Bank/checkpoints/chatbot_cpu/model.cpkt-723')

INFO:tensorflow:Restoring parameters from 109_Bank/checkpoints/chatbot_cpu/model.cpkt-723


In [10]:
cpu_saver.save(session, '109_Bank/checkpoints/chatbot_cpu/model.cpkt-723')

'109_Bank/checkpoints/chatbot_cpu/model.cpkt-723'

In [21]:
output_graph_def = tf.graph_util.convert_variables_to_constants(
    session, tf.get_default_graph().as_graph_def(),
    [classifier.probs.op.name, tagger.decode_sequence.op.name, tagger.decode_score.op.name]
)

INFO:tensorflow:Froze 54 variables.
INFO:tensorflow:Converted 54 variables to const ops.


In [23]:
!mkdir -p 109_Bank/checkpoints/chatbot_frozen/
with tf.gfile.GFile('109_Bank/checkpoints/chatbot_frozen/frozen.pb-723', "wb") as f:
    f.write(output_graph_def.SerializeToString())

In [ ]:
[classifier.probs.op.name, tagger.decode_sequence.op.name, tagger.decode_score.op.name]